# RecBole Model Training - Real Estate Session-Based Recommendation

Este notebook treina modelos de recomendação sequencial usando RecBole com dados de sessão de imóveis.

**Baseado em:** [RecBole Sequential Model Tutorial](https://github.com/RUCAIBox/RecBole/blob/master/run_example/sequential-model-fixed-missing-last-item.ipynb)

## Estrutura

1. Setup e Importações
2. Configuração do Ambiente
3. **Criação do Dataset e Treinamento do Modelo**
4. Avaliação e Predição
5. Análise dos Resultados

## 1. Setup e Importações

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

print(f"Project root: {project_root}")
print(f"Working directory: {Path.cwd()}")

In [ ]:
import logging
import yaml
import numpy as np
import pandas as pd
import torch
from logging import getLogger

# Fix PyTorch 2.6+ weights_only compatibility issue
_original_torch_load = torch.load
def _patched_torch_load(*args, **kwargs):
    kwargs.setdefault('weights_only', False)
    return _original_torch_load(*args, **kwargs)
torch.load = _patched_torch_load

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, NARM, SASRec, STAMP
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")

## 2. Configuração do Ambiente

In [ ]:
from pathlib import Path

MODEL_NAME = 'GRU4Rec'
DATASET_NAME = 'realestate_simple'
GPU_ID = 0

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()

DATA_PATH = PROJECT_ROOT / 'outputs' / 'data' / 'recbole_simple'
CHECKPOINT_DIR = PROJECT_ROOT / 'outputs' / 'saved'


CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Data path: {DATA_PATH}")
print(f"Checkpoint dir: {CHECKPOINT_DIR}")

## 3. Criação do Dataset e Treinamento do Modelo

Esta seção implementa o passo 3 do notebook original: **"Create dataset and train model with Recbole"**

### 3.1 Carregar Configurações

In [ ]:
# Configuração inline do GRU4Rec
config_dict = {
    'model': 'GRU4Rec',
    'dataset': DATASET_NAME,
    'data_path': str(DATA_PATH),

    # Training
    'epochs': 10,
    'train_batch_size': 2048,
    'eval_batch_size': 2048,
    'learning_rate': 0.001,
    'train_neg_sample_args': None,

    # Early Stopping
    'stopping_step': 2,

    # Model Parameters
    'embedding_size': 64,
    'hidden_size': 128,
    'num_layers': 2,
    'dropout_prob': 0.3,
    'loss_type': 'CE',

    # Gradient Clipping
    'clip_grad_norm': {'max_norm': 5.0},

    # Evaluation
    'metrics': ['Recall', 'MRR', 'NDCG', 'Hit'],
    'topk': [5, 10, 20],
    'valid_metric': 'MRR@10',

    # Session Settings
    'MAX_ITEM_LIST_LENGTH': 50,
    'SESSION_ID_FIELD': 'session_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'USER_ID_FIELD': 'session_id',
    'load_col': {
        'inter': ['session_id', 'item_id', 'timestamp']
    },

    # Evaluation Protocol
    'eval_args': {
        'split': {'LS': 'valid_and_test'},
        'order': 'TO',
        'mode': 'full'
    },

    # Device
    'device': 'cuda',
    'gpu_id': GPU_ID,

    # Reproducibility
    'seed': 42,

    # Checkpointing
    'save_dataset': False,
    'save_dataloaders': False,
    'checkpoint_dir': str(CHECKPOINT_DIR),

    # Logging
    'log_wandb': False,
}

print("\nConfiguração carregada:")
print(f"  Model: {config_dict['model']}")
print(f"  Dataset: {config_dict['dataset']}")
print(f"  Epochs: {config_dict['epochs']}")
print(f"  Batch size (train): {config_dict['train_batch_size']}")
print(f"  Batch size (eval): {config_dict['eval_batch_size']}")
print(f"  Learning rate: {config_dict['learning_rate']}")
print(f"  Embedding size: {config_dict['embedding_size']}")
print(f"  Hidden size: {config_dict['hidden_size']}")
print(f"  Max sequence length: {config_dict['MAX_ITEM_LIST_LENGTH']}")

### 3.2 Inicializar RecBole Config

In [ ]:
# Create RecBole config
config = Config(
    model=config_dict['model'],
    dataset=config_dict['dataset'],
    config_dict=config_dict
)

# Initialize random seed for reproducibility
init_seed(config['seed'], config['reproducibility'])

# Initialize logger
init_logger(config)
logger = getLogger()

# Create handler for notebook output
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# Log configuration
logger.info(config)

### 3.3 Criar Dataset

In [7]:
# Create dataset
print("\n" + "="*80)
print("CREATING DATASET")
print("="*80 + "\n")

dataset = create_dataset(config)
logger.info(dataset)

print(f"\nDataset Statistics:")
print(f"  Number of sessions: {dataset.user_num}")
print(f"  Number of items: {dataset.item_num}")
print(f"  Number of interactions: {dataset.inter_num}")
print(f"  Average session length: {dataset.inter_num / dataset.user_num:.2f}")
print(f"  Sparsity: {100 * (1 - dataset.inter_num / (dataset.user_num * dataset.item_num)):.4f}%")


CREATING DATASET



/home/hygo2025/Development/projects/fermi/.venv/lib/python3.9/site-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/home/hygo2025/Development/projects/fermi/.venv/lib/python3.9/site-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the 


Dataset Statistics:
  Number of sessions: 174122
  Number of items: 40033
  Number of interactions: 1212852
  Average session length: 6.97
  Sparsity: 99.9826%


### 3.4 Preparar Dados de Treinamento e Avaliação

In [8]:
# Split dataset
print("\n" + "="*80)
print("PREPARING DATA SPLITS")
print("="*80 + "\n")

train_data, valid_data, test_data = data_preparation(config, dataset)

print(f"\nData Split:")
print(f"  Train batches: {len(train_data)}")
print(f"  Valid batches: {len(valid_data) if valid_data else 0}")
print(f"  Test batches: {len(test_data) if test_data else 0}")


PREPARING DATA SPLITS



18 Dec 10:47    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
18 Dec 10:47    INFO  [Evaluation]: eval_batch_size = [2048] eval_args: [{'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]
[Evaluation]: eval_batch_size = [2048] eval_args: [{'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]



Data Split:
  Train batches: 400
  Valid batches: 50
  Test batches: 58


### 3.5 Inicializar Modelo

In [9]:
# Map model name to class
MODEL_MAP = {
    'GRU4Rec': GRU4Rec,
    'NARM': NARM,
    'SASRec': SASRec,
    'STAMP': STAMP
}

# Initialize model
print("\n" + "="*80)
print("INITIALIZING MODEL")
print("="*80 + "\n")

model_class = MODEL_MAP.get(MODEL_NAME)
if model_class is None:
    raise ValueError(f"Model {MODEL_NAME} not supported. Choose from: {list(MODEL_MAP.keys())}")

model = model_class(config, train_data.dataset).to(config['device'])
logger.info(model)

# Count trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nModel Architecture:")
print(f"  Total trainable parameters: {trainable_params:,}")
print(f"  Device: {config['device']}")


INITIALIZING MODEL



18 Dec 10:48    INFO  GRU4Rec(
  (item_embedding): Embedding(40033, 64, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(64, 128, num_layers=2, bias=False, batch_first=True)
  (dense): Linear(in_features=128, out_features=64, bias=True)
  (loss_fct): CrossEntropyLoss()
)
Trainable parameters: 2742400
GRU4Rec(
  (item_embedding): Embedding(40033, 64, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(64, 128, num_layers=2, bias=False, batch_first=True)
  (dense): Linear(in_features=128, out_features=64, bias=True)
  (loss_fct): CrossEntropyLoss()
)
Trainable parameters: 2742400



Model Architecture:
  Total trainable parameters: 2,742,400
  Device: cuda


### 3.6 Treinar Modelo

In [10]:
# Initialize trainer
trainer = Trainer(config, model)

print("\n" + "="*80)
print("STARTING MODEL TRAINING")
print("="*80 + "\n")

# Train model
best_valid_score, best_valid_result = trainer.fit(
    train_data, 
    valid_data,
    saved=True,
    show_progress=True
)

print("\n" + "="*80)
print("TRAINING COMPLETED")
print("="*80 + "\n")
print(f"Best validation score: {best_valid_score:.4f}")
print(f"Best validation result: {best_valid_result}")


STARTING MODEL TRAINING



Train     0:   0%|                                                          | 0/400 [00:00<?, ?it/s]/home/hygo2025/Development/projects/fermi/.venv/lib/python3.9/site-packages/recbole/trainer/trainer.py:235: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=self.enable_scaler)
Train     0:   0%|                                 | 0/400 [00:00<?, ?it/s, GPU RAM: 2.79 G/23.64 G]:   0%|                         | 1/400 [00:00<02:02,  3.27it/s, GPU RAM: 2.79 G/23.64 G]:   0%|                         | 1/400 [00:00<02:02,  3.27it/s, GPU RAM: 2.79 G/23.64 G]:   0%|                         | 1/400 [00:00<02:02,  3.27it/s, GPU RAM: 2.79 G/23.64 G]:   0%|                         | 1/400 [00:00<02:02,  3.27it/s, GPU RAM: 2.79 G/23.64 G]:   0%|                         | 1/400 [00:00<02:02,  3.27it/s, GPU RAM: 2.79 G/23.64 G]:   1%|▎                        | 5/400 [00:00<00:26, 14.90it/s, G


TRAINING COMPLETED

Best validation score: 0.2737
Best validation result: OrderedDict([('recall@5', 0.3298), ('recall@10', 0.3834), ('recall@20', 0.4449), ('mrr@5', 0.2666), ('mrr@10', 0.2737), ('mrr@20', 0.2779), ('ndcg@5', 0.2823), ('ndcg@10', 0.2996), ('ndcg@20', 0.3151), ('hit@5', 0.3298), ('hit@10', 0.3834), ('hit@20', 0.4449)])


## 4. Avaliação e Predição

### 4.1 Avaliar no Test Set

In [11]:
if test_data:
    print("\n" + "="*80)
    print("EVALUATING ON TEST SET")
    print("="*80 + "\n")
    
    test_result = trainer.evaluate(test_data, load_best_model=True, show_progress=True)
    
    print(f"\nTest Results:")
    for metric, value in test_result.items():
        print(f"  {metric}: {value:.4f}")
else:
    print("\nNo test data available.")


EVALUATING ON TEST SET



18 Dec 10:52    INFO  Loading model structure and parameters from /home/hygo2025/Development/projects/fermi/outputs/saved/GRU4Rec-Dec-18-2025_10-48-31.pth
Loading model structure and parameters from /home/hygo2025/Development/projects/fermi/outputs/saved/GRU4Rec-Dec-18-2025_10-48-31.pth
Evaluate   :   0%|                                                           | 0/58 [00:00<?, ?it/s]:   0%|                                  | 0/58 [00:00<?, ?it/s, GPU RAM: 2.79 G/23.64 G]:   0%|                                  | 0/58 [00:00<?, ?it/s, GPU RAM: 2.79 G/23.64 G]:   0%|                                  | 0/58 [00:00<?, ?it/s, GPU RAM: 2.79 G/23.64 G]:   0%|                                  | 0/58 [00:00<?, ?it/s, GPU RAM: 2.79 G/23.64 G]:   0%|                                  | 0/58 [00:00<?, ?it/s, GPU RAM: 2.79 G/23.64 G]:   0%|                                  | 0/58 [00:00<?, ?it/s, GPU RAM: 2.79 G/23.64 G]:   0%|                                  | 0/58 [00:00<?, ?it/s, GPU RAM: 2.79


Test Results:
  recall@5: 0.7377
  recall@10: 0.7671
  recall@20: 0.7956
  mrr@5: 0.6795
  mrr@10: 0.6834
  mrr@20: 0.6854
  ndcg@5: 0.6941
  ndcg@10: 0.7037
  ndcg@20: 0.7109
  hit@5: 0.7377
  hit@10: 0.7671
  hit@20: 0.7956


### 4.2 Exemplo de Predição para Sessões

In [13]:
from recbole.data.interaction import Interaction

def predict_for_session(session_id, dataset, model, topk=10):
    """
    Faz predição para uma sessão específica
    
    Args:
        session_id: ID externo da sessão
        dataset: RecBole dataset object
        model: Modelo treinado
        topk: Número de recomendações
    
    Returns:
        scores, item_ids: Scores e IDs dos itens recomendados
    """
    model.eval()
    with torch.no_grad():
        # Convert external ID to internal ID
        uid_series = dataset.token2id(dataset.uid_field, [session_id])
        
        # Get session interactions
        index = np.isin(dataset[dataset.uid_field].numpy(), uid_series)
        session_data = dataset[index]
        
        if len(session_data) == 0:
            print(f"Session {session_id} not found in dataset")
            return None, None
        
        # Prepare input for prediction
        # Get the last item in the session
        last_item = session_data['item_id'][-1].item()
        
        # Create interaction for prediction
        item_seq = session_data['item_id_list'][-1].unsqueeze(0)
        item_len = session_data['item_length'][-1].unsqueeze(0)
        
        test_input = {
            'item_id_list': item_seq,
            'item_length': item_len
        }
        
        new_inter = Interaction(test_input)
        new_inter = new_inter.to(config['device'])
        
        # Get predictions
        scores = model.full_sort_predict(new_inter)
        scores = scores.view(-1, dataset.item_num)
        scores[:, 0] = -np.inf  # Remove padding token
        
        # Get top-k items
        topk_scores, topk_iids = torch.topk(scores, topk)
        
    return topk_scores, topk_iids

print("Prediction function defined.")

Prediction function defined.


In [14]:
# Get some example sessions
external_session_ids = dataset.id2token(
    dataset.uid_field, list(range(1, 6))
)

print("\n" + "="*80)
print("EXAMPLE PREDICTIONS")
print("="*80 + "\n")

for session_id in external_session_ids:
    scores, item_ids = predict_for_session(session_id, dataset, model, topk=10)
    
    if scores is not None:
        # Convert internal item IDs to external IDs
        external_items = dataset.id2token(
            dataset.iid_field, 
            item_ids[0].cpu().tolist()
        )
        
        print(f"\nSession: {session_id}")
        print(f"  Top-10 Recommended Items:")
        for i, (item, score) in enumerate(zip(external_items, scores[0].cpu().tolist()), 1):
            print(f"    {i}. Item {item}: {score:.4f}")


EXAMPLE PREDICTIONS


Session: S_1000741
  Top-10 Recommended Items:
    1. Item 18615: 28.6344
    2. Item 164184: 25.4981
    3. Item 27314: 24.9688
    4. Item 173416: 24.9062
    5. Item 80105: 24.9049
    6. Item 48341: 24.4223
    7. Item 109611: 24.3741
    8. Item 85284: 24.1846
    9. Item 177064: 24.0847
    10. Item 106457: 23.9603

Session: S_1000747
  Top-10 Recommended Items:
    1. Item 41168: 28.4635
    2. Item 50796: 26.2039
    3. Item 133741: 25.4983
    4. Item 79740: 24.8706
    5. Item 111630: 24.6173
    6. Item 37937: 24.4354
    7. Item 172993: 24.2002
    8. Item 71146: 24.1596
    9. Item 61450: 24.1380
    10. Item 103282: 24.1015

Session: S_1000755
  Top-10 Recommended Items:
    1. Item 85791: 33.1281
    2. Item 80188: 26.7127
    3. Item 160002: 26.4123
    4. Item 96639: 25.6694
    5. Item 17780: 24.6984
    6. Item 93890: 24.6407
    7. Item 182372: 24.2337
    8. Item 169488: 24.0796
    9. Item 125507: 24.0660
    10. Item 111941: 23.8190

Sessio

### 4.3 Gerar Predições em Lote

In [15]:
def generate_recommendations_for_all_sessions(dataset, model, topk=10, batch_size=100):
    """
    Gera recomendações para todas as sessões no dataset
    
    Args:
        dataset: RecBole dataset object
        model: Modelo treinado
        topk: Número de recomendações
        batch_size: Tamanho do lote para processamento
    
    Returns:
        DataFrame com session_id e lista de itens recomendados
    """
    model.eval()
    
    # Get all external session IDs
    all_sessions = dataset.id2token(
        dataset.uid_field, 
        list(range(1, dataset.user_num))  # Skip padding token at 0
    )
    
    results = []
    
    print(f"Generating recommendations for {len(all_sessions)} sessions...")
    
    for i in range(0, len(all_sessions), batch_size):
        batch_sessions = all_sessions[i:i+batch_size]
        
        for session_id in batch_sessions:
            _, item_ids = predict_for_session(session_id, dataset, model, topk=topk)
            
            if item_ids is not None:
                # Convert to external IDs
                external_items = dataset.id2token(
                    dataset.iid_field, 
                    item_ids[0].cpu().tolist()
                )
                
                results.append({
                    'session_id': session_id,
                    'recommendations': ' '.join(map(str, external_items))
                })
        
        if (i // batch_size + 1) % 10 == 0:
            print(f"  Processed {i + len(batch_sessions)}/{len(all_sessions)} sessions...")
    
    return pd.DataFrame(results)

print("Batch prediction function defined.")

Batch prediction function defined.


In [16]:
# Generate recommendations for first 1000 sessions as example
# Remove the slice to generate for all sessions (will take longer)
sample_sessions = dataset.id2token(
    dataset.uid_field, 
    list(range(1, min(1001, dataset.user_num)))
)

print(f"\nGenerating recommendations for {len(sample_sessions)} sample sessions...")

recommendations_df = pd.DataFrame()
for session_id in sample_sessions:
    _, item_ids = predict_for_session(session_id, dataset, model, topk=10)
    if item_ids is not None:
        external_items = dataset.id2token(
            dataset.iid_field, 
            item_ids[0].cpu().tolist()
        )
        recommendations_df = pd.concat([
            recommendations_df,
            pd.DataFrame({
                'session_id': [session_id],
                'recommendations': [' '.join(map(str, external_items))]
            })
        ], ignore_index=True)

print(f"\nGenerated recommendations for {len(recommendations_df)} sessions")
print(f"\nSample recommendations:")
print(recommendations_df.head(10))


Generating recommendations for 1000 sample sessions...

Generated recommendations for 1000 sessions

Sample recommendations:
  session_id                                    recommendations
0  S_1000741  18615 164184 27314 173416 80105 48341 109611 8...
1  S_1000747  41168 50796 133741 79740 111630 37937 172993 7...
2  S_1000755  85791 80188 160002 96639 17780 93890 182372 16...
3  S_1000775  146882 80266 96024 180736 3404 144092 47344 13...
4  S_1000778  93013 185363 29788 162758 150829 149495 79942 ...
5  S_1000779  54933 53961 177942 94389 119122 57611 52997 33...
6  S_1000783  156523 2271 181572 80519 1239 60946 16920 791 ...
7  S_1000787  112498 164768 94664 9643 46106 98194 180786 18...
8  S_1000789  168660 147987 111554 49603 143690 145052 13661...
9  S_1000799  64162 48304 93527 31891 94759 147987 60095 773...


### 4.4 Salvar Recomendações

In [ ]:
# Save recommendations to CSV
output_file = project_root / 'outputs' / 'recommendations' / f'{MODEL_NAME}_{DATASET_SLICE}_recommendations.csv'
output_file.parent.mkdir(parents=True, exist_ok=True)

recommendations_df.to_csv(output_file, index=False)
print(f"\nRecommendations saved to: {output_file}")

## 5. Análise dos Resultados

### 5.1 Estatísticas do Treinamento

In [ ]:
import matplotlib.pyplot as plt

# Note: RecBole doesn't directly expose training history
# You would need to modify the Trainer to save epoch-wise metrics
# This is a placeholder for visualization

print("\n" + "="*80)
print("TRAINING SUMMARY")
print("="*80 + "\n")

print(f"Model: {MODEL_NAME}")
print(f"Dataset: {config['dataset']}")
print(f"Total epochs trained: {config['epochs']}")
print(f"Best validation metric ({config['valid_metric']}): {best_valid_score:.4f}")
print(f"\nBest validation results:")
for metric, value in best_valid_result.items():
    print(f"  {metric}: {value:.4f}")

if test_data:
    print(f"\nTest results:")
    for metric, value in test_result.items():
        print(f"  {metric}: {value:.4f}")

### 5.2 Distribuição das Recomendações

In [19]:
# Analyze recommendation distribution
if len(recommendations_df) > 0:
    # Extract all recommended items
    all_recommended_items = []
    for recs in recommendations_df['recommendations']:
        all_recommended_items.extend(recs.split())
    
    # Count frequency
    from collections import Counter
    item_counts = Counter(all_recommended_items)
    
    print(f"\nRecommendation Statistics:")
    print(f"  Total recommendations: {len(all_recommended_items)}")
    print(f"  Unique items recommended: {len(item_counts)}")
    print(f"  Coverage: {100 * len(item_counts) / dataset.item_num:.2f}%")
    
    print(f"\nTop 10 most recommended items:")
    for item, count in item_counts.most_common(10):
        print(f"  Item {item}: {count} times ({100*count/len(recommendations_df):.1f}% of sessions)")


Recommendation Statistics:
  Total recommendations: 10000
  Unique items recommended: 7032
  Coverage: 17.57%

Top 10 most recommended items:
  Item 4769: 12 times (1.2% of sessions)
  Item 11605: 11 times (1.1% of sessions)
  Item 80188: 10 times (1.0% of sessions)
  Item 93890: 10 times (1.0% of sessions)
  Item 49603: 10 times (1.0% of sessions)
  Item 137382: 10 times (1.0% of sessions)
  Item 33732: 10 times (1.0% of sessions)
  Item 93527: 9 times (0.9% of sessions)
  Item 167559: 9 times (0.9% of sessions)
  Item 139506: 9 times (0.9% of sessions)


### 5.3 Informações do Checkpoint

In [20]:
# Find saved checkpoint
checkpoint_pattern = f"{MODEL_NAME}-*.pth"
checkpoint_files = list(CHECKPOINT_DIR.glob(checkpoint_pattern))

if checkpoint_files:
    latest_checkpoint = max(checkpoint_files, key=lambda p: p.stat().st_mtime)
    checkpoint_size = latest_checkpoint.stat().st_size / (1024 * 1024)  # MB
    
    print(f"\nModel Checkpoint:")
    print(f"  File: {latest_checkpoint.name}")
    print(f"  Size: {checkpoint_size:.2f} MB")
    print(f"  Path: {latest_checkpoint}")
else:
    print(f"\nNo checkpoint found matching pattern: {checkpoint_pattern}")


Model Checkpoint:
  File: GRU4Rec-Dec-18-2025_10-48-31.pth
  Size: 31.40 MB
  Path: /home/hygo2025/Development/projects/fermi/outputs/saved/GRU4Rec-Dec-18-2025_10-48-31.pth


## 6. Próximos Passos

Possíveis extensões deste notebook:

1. **Experimentar outros modelos**: NARM, SASRec, STAMP
2. **Tunning de hiperparâmetros**: Grid search ou Optuna
3. **Análise de erro**: Investigar sessões com baixa performance
4. **Ensemble**: Combinar predições de múltiplos modelos
5. **Cold-start**: Estratégias para sessões novas
6. **Feature engineering**: Adicionar features de itens (se disponíveis)
7. **Explicabilidade**: Analisar attention weights (para modelos que suportam)

Para treinar outros modelos ou slices, apenas mude as variáveis `MODEL_NAME` e `DATASET_SLICE` na seção 2.

In [ ]:
print("\n" + "="*80)
print("NOTEBOOK COMPLETED SUCCESSFULLY")
print("="*80)